In [102]:
import urllib.request
import ast
from bs4 import BeautifulSoup as bs
import requests

def find_word(client_id,client_secret,keyword):
    encText = urllib.parse.quote(keyword)
    url = "https://openapi.naver.com/v1/search/encyc?query=" + encText +"&display=3&sort=count"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        mydict = ast.literal_eval(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)
    
    for i in mydict['items']:
        # 법률용어사전을 우선적으로 탐색
        if '=42131&' in i['link']:
            link = i['link']
            break
    # 없다면 두산백과 탐색
    else:
        for k in mydict['items']:
            if 'cid=40942' in k['link']:
                link = k['link']
                break
        # 둘 다 없다면 Not found 반환
        else:
            return 'Not found'
        
    # 받은 링크를 통해 웹 크롤링으로 전체 설명 반환 
    des_req = requests.get(link)
    des_bs = bs(des_req.text, 'html')
    description = des_bs.find('meta', property='og:description')['content'] + '\n\n' + '출처: {}'.format(link)
    return description


In [106]:
find_word(client_id,client_secret,'피고인')

피고인이라 함은 검사에 의하여 형사책임을 져야 할 자로 공소가 제기된 자 또는 공소가 제기된 자로 취급되어 있는 자를 말한다.피고인은 공소제기 이후의 개념이므로 수사기관에 의하여 수사의 대상으로 되어 있는 피의자와 구별되며 또한 판결확정 이전의 개념이므로 확정판결에 의하여 형의 집행을 받고 있는 수형인(受刑人)과 구별된다. 누구를 피고인으로 보아야 할 것인가는 공소장에 기재된 자인가의 여부에 따라서 판정된다는 견해(공시설(公示說))와 이에 대하여 피고인으로서 실제로 행위를 한 자를 피고인이라고 보는 설(행위설(行爲說)), 그리고 검사의 의사를 기준으로 하여 피고인을 결정하여야 한다는 설(의사설(意思說)) 등이 대립하고 있는데 형사소송에서는 전(前) 2설을 포괄하여 결정함이 타당하다는 것이 통설이다. 이러한 피고인은 판결이 확정되어 유죄로 확정될 때까지는 무죄로 추정되며 그러므로 검사가 충분하게 입증하지 못한 때에는 비록 혐의가 있더라도 범인이...

출처: https://terms.naver.com/entry.nhn?docId=3654608&cid=42131&categoryId=42131
